<a href="https://colab.research.google.com/github/harjeet88/LLM_experiemnts/blob/main/LLMs_course/week1/ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]


In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [9]:
!curl -LO https://ollama.ai/install.sh
!bash install.sh



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  62299      0 --:--:-- --:--:-- --:--:-- 62352
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [14]:
!ollama serve

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAICQEwr7LUcngSRf7y9bB0wUziv0/Gy06Q6FhfITs1rWH

2025/03/10 07:59:45 routes.go:1215: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:2048 OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://

In [15]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

Error: could not connect to ollama app, is it running?


In [ ]:
def ingest_documents(document_paths, index, texts):
    """Ingest documents into the vector database."""
    for doc_path in document_paths:
        try:
            with open(doc_path, "r", encoding="utf-8") as f:
                text = f.read()
                texts.append(text)
        except FileNotFoundError:
            print(f"File not found: {doc_path}")
        except Exception as e:
            print(f"Error processing {doc_path}: {e}")

    embeddings = get_embeddings(texts[-len(document_paths):]) #generate only the new embeddings.
    index.add(embeddings)
    print("Documents ingested successfully.")


In [ ]:
def retrieve_relevant_context(question, index, texts):
    """Retrieve relevant context from the vector database."""
    question_embedding = get_embeddings([question])
    D, I = index.search(question_embedding, 5) # Search top 5
    relevant_texts = [texts[i] for i in I[0]]
    return " ".join(relevant_texts)

In [ ]:
def generate_answer(context, question):
    """Generate an answer using the Flan-T5 model."""
    input_text = f"context: {context} question: {question}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=50)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


In [ ]:
def rag_pipeline(question, index, texts):
    """RAG pipeline using FAISS and Flan-T5."""
    context = retrieve_relevant_context(question, index, texts)
    if context:
        answer = generate_answer(context, question)
        return answer
    else:
        return "No relevant information found."


In [ ]:
# Example usage:
texts = [] # List to store document texts
embeddings_dim = 768  # Dimension of embeddings from all-mpnet-base-v2
index = faiss.IndexFlatL2(embeddings_dim) # Create FAISS index


In [ ]:
# Create example documents (for testing)
#doc1_content = "The Eiffel Tower is in Paris."
#doc2_content = "The capital of Japan is Tokyo."
#doc3_content = "Python is a popular programming language."

# HP Usage

doc1_content = "Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The novels chronicle the lives of a young wizard, Harry Potter, and his friends, Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns Harry's conflict with Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the Ministry of Magic, and subjugate all wizards and Muggles (non-magical people)."
doc2_content="Harry learns that his parents, Lily and James Potter, also had magical powers and were murdered by the dark wizard Lord Voldemort when Harry was a baby."
doc3_content="He gains the friendship of Ron Weasley, a member of a large but poor wizarding family, and Hermione Granger, a witch of non-magical, or Muggle, parentage."
doc4_content="he first book concludes with Harry's confrontation with Voldemort, who, in his quest to regain a body, yearns to possess the Philosopher's Stone, a substance that bestows everlasting life."
doc5_content="Tom riddle is actual Voldemort. Harry learns from a drunken Slughorn that he used to teach Tom Riddle, and that Voldemort divided his soul into pieces, creating a series of Horcruxes."




In [ ]:
with open("doc1.txt", "w", encoding="utf-8") as f:
    f.write(doc1_content)
with open("doc2.txt", "w", encoding="utf-8") as f:
    f.write(doc2_content)
with open("doc3.txt", "w", encoding="utf-8") as f:
    f.write(doc3_content)
with open("doc4.txt", "w", encoding="utf-8") as f:
    f.write(doc5_content)
with open("doc5.txt", "w", encoding="utf-8") as f:
    f.write(doc5_content)

In [ ]:
document_paths = ["doc1.txt", "doc2.txt", "doc3.txt"]
ingest_documents(document_paths, index, texts)

Documents ingested successfully.


In [ ]:
question = "who are friends of harry potter?"
answer = rag_pipeline(question, index, texts)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: who are friends of harry potter?
Answer: Hermione Granger and Ron Weasley


In [ ]:
question = "who is enemy of harry potter"
answer = rag_pipeline(question, index, texts)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: who is enemy of harry potter
Answer: Lord Voldemort
